<a href="https://colab.research.google.com/github/jasminjahanpuspo/CNN_Architectures_Template/blob/main/5K_fold_EnsembleNet(3_poor_model).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import load_model
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold
from tensorflow.keras.preprocessing import image_dataset_from_directory

In [2]:
import numpy as np
from keras.models import Model, load_model
from keras.layers import Input, Dense
from keras.preprocessing import image_dataset_from_directory
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score


In [3]:
import numpy as np
import cv2
import os
import pandas as pd

import glob as gb
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

import tensorflow as tf
from tensorflow import keras

from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras.layers.experimental.preprocessing import RandomFlip, RandomRotation
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.layers import Conv2D, MaxPool2D ,LeakyReLU
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Average

In [4]:
# Set your parameters
BATCH_SIZE = 32
IMG_SIZE = (224, 224)  # Adjust this according to your model requirements
N_FOLDS = 5

In [5]:
## Define the directories for training, testing, and validation
train_directory = '/content/drive/MyDrive/MIAS/train'
test_directory = '/content/drive/MyDrive/MIAS/test'
valid_directory = '/content/drive/MyDrive/MIAS/val'

train_dataset = image_dataset_from_directory(
    train_directory,
    shuffle=True,
    labels='inferred',
    batch_size=32,
    image_size=(224, 224),
    color_mode='rgb',
    seed=42
)

test_dataset = image_dataset_from_directory(
    test_directory,
    shuffle=False,
    labels='inferred',
    batch_size=32,
    image_size=(224, 224),
    color_mode='rgb',
    seed=42
)

val_dataset = image_dataset_from_directory(
    valid_directory,
    shuffle=True,
    labels='inferred',
    batch_size=32,
    image_size=(224, 224),
    color_mode='rgb',
    seed=42
)

valid_dataset = image_dataset_from_directory(
    valid_directory,
    shuffle=True,
    labels='inferred',
    batch_size=32,
    image_size=(224, 224),
    color_mode='rgb',
    seed=42
)

Found 3294 files belonging to 2 classes.
Found 573 files belonging to 2 classes.
Found 573 files belonging to 2 classes.
Found 573 files belonging to 2 classes.


In [6]:
# Load pre-trained models
model_1 = load_model('/content/drive/MyDrive/MIAS/BC_Code/resnet50/resnet50-24-0.8534.hdf5')
model_2 = load_model('/content/drive/MyDrive/MIAS/BC_Code/densenet121/densenet121-20-0.9162.hdf5')
model_3 = load_model('/content/drive/MyDrive/MIAS/BC_Code/MobileNetv3/mobilenetv3-29-0.8080.hdf5')


In [7]:
# Function to get predictions from a model
def get_predictions(model, data):
    predictions = model.predict(data)
    return np.argmax(predictions, axis=1)

# Function to get average predictions from multiple models
def get_ensemble_predictions(models, data):
    # Get predictions from each model
    predictions = [model.predict(data) for model in models]

    # Average the predictions
    avg_predictions = np.mean(predictions, axis=0)

    # Convert averaged predictions to class labels
    return np.argmax(avg_predictions, axis=1)


In [8]:
# Assuming train_dataset and val_dataset are preprocessed datasets
# Replace with your own data pipeline
N_FOLDS = 5  # Define number of folds

# Prepare for K-Fold Cross-Validation
kf = KFold(n_splits=N_FOLDS, shuffle=True, random_state=42)

# Store accuracy for each fold for the ensemble
ensemble_accuracies = []


In [9]:
# K-Fold Cross-Validation
for fold, (train_index, val_index) in enumerate(kf.split(np.arange(len(train_dataset)))):
    print(f'Training on fold {fold + 1}...')

    # Create train and validation datasets based on K-Fold indices
    train_fold = train_dataset.skip(val_index[0]).take(len(train_index))
    val_fold = train_dataset.skip(train_index[0]).take(len(val_index))

    # Get true labels for validation data
    y_val = []
    for _, labels in val_fold:
        y_val.extend(labels.numpy())

    y_val = np.array(y_val)

    # Train all models on the training fold
    history_model_1 = model_1.fit(
        train_fold,
        epochs=5,  # Adjust epochs as necessary
        validation_data=val_fold,
        verbose=2
    )

    history_model_2 = model_2.fit(
        train_fold,
        epochs=5,
        validation_data=val_fold,
        verbose=2
    )

    history_model_3 = model_3.fit(
        train_fold,
        epochs=5,
        validation_data=val_fold,
        verbose=2
    )

    # Get ensemble predictions for the validation fold
    models = [model_1, model_2, model_3]
    ensemble_predictions = get_ensemble_predictions(models, val_fold)

    # Calculate accuracy for the ensemble model
    ensemble_accuracy = accuracy_score(y_val, ensemble_predictions)
    ensemble_accuracies.append(ensemble_accuracy)

    print(f'Fold {fold + 1} Ensemble Accuracy: {ensemble_accuracy:.4f}')

# Calculate mean accuracy across all folds for the ensemble
mean_ensemble_accuracy = np.mean(ensemble_accuracies)
print(f'Mean K-Fold Ensemble Accuracy: {mean_ensemble_accuracy:.4f}')

Training on fold 1...
Epoch 1/5
82/82 - 68s - loss: 0.3390 - accuracy: 0.8540 - val_loss: 0.2230 - val_accuracy: 0.9271 - 68s/epoch - 827ms/step
Epoch 2/5
82/82 - 57s - loss: 0.3481 - accuracy: 0.8434 - val_loss: 0.2126 - val_accuracy: 0.9449 - 57s/epoch - 699ms/step
Epoch 3/5
82/82 - 56s - loss: 0.3314 - accuracy: 0.8575 - val_loss: 0.2171 - val_accuracy: 0.9256 - 56s/epoch - 682ms/step
Epoch 4/5
82/82 - 56s - loss: 0.3270 - accuracy: 0.8579 - val_loss: 0.1943 - val_accuracy: 0.9390 - 56s/epoch - 682ms/step
Epoch 5/5
82/82 - 57s - loss: 0.3206 - accuracy: 0.8601 - val_loss: 0.2047 - val_accuracy: 0.9256 - 57s/epoch - 697ms/step
Epoch 1/5
82/82 - 91s - loss: 0.2946 - accuracy: 0.8883 - val_loss: 0.1843 - val_accuracy: 0.9479 - 91s/epoch - 1s/step
Epoch 2/5
82/82 - 73s - loss: 0.1490 - accuracy: 0.9459 - val_loss: 0.2851 - val_accuracy: 0.8765 - 73s/epoch - 892ms/step
Epoch 3/5
82/82 - 73s - loss: 0.1376 - accuracy: 0.9573 - val_loss: 0.5948 - val_accuracy: 0.8110 - 73s/epoch - 888ms/st

In [10]:
# Evaluate on the test dataset
models = [model_1, model_2, model_3]
test_predictions = get_ensemble_predictions(models, test_dataset)

# Get true labels for the test dataset
y_test = []
for _, labels in test_dataset:
    y_test.extend(labels.numpy())

y_test = np.array(y_test)

# Calculate accuracy for the test dataset
test_accuracy = accuracy_score(y_test, test_predictions)
print(f'Test Accuracy: {test_accuracy:.4f}')

18/18 [==============================] - 1s 68ms/step
Test Accuracy: 0.9703


In [11]:
# Get true labels for the validation or test set
y_true = []
for _, labels in val_fold:  # Change val_fold to test_dataset for test data
    y_true.extend(labels.numpy())

y_true = np.array(y_true)  # Convert to numpy array for easier processing


In [12]:
def get_ensemble_predictions(models, dataset):
    # Create an empty list to store predictions from all models
    all_predictions = []

    for model in models:
        model_predictions = []
        for images, _ in dataset:  # Loop through dataset without using labels
            predictions = model.predict(images)
            model_predictions.append(predictions)
        all_predictions.append(np.concatenate(model_predictions))

    # Average the predictions from all models (for classification)
    averaged_predictions = np.mean(np.array(all_predictions), axis=0)

    # Convert averaged predictions into class labels (assuming softmax probabilities)
    predicted_labels = np.argmax(averaged_predictions, axis=1)

    return predicted_labels


In [13]:
y_pred = get_ensemble_predictions(models, val_fold)  # or test_dataset for test set


1/1 [==============================] - 0s 81ms/step


In [14]:
from sklearn.metrics import confusion_matrix, classification_report

# Confusion matrix
conf_matrix = confusion_matrix(y_true, y_pred)
print('Confusion Matrix:')
print(conf_matrix)

# Detailed classification report (Precision, Recall, F1-score)
report = classification_report(y_true, y_pred)
print('Classification Report:')
print(report)


Confusion Matrix:
[[193 145]
 [182 120]]
Classification Report:
              precision    recall  f1-score   support

           0       0.51      0.57      0.54       338
           1       0.45      0.40      0.42       302

    accuracy                           0.49       640
   macro avg       0.48      0.48      0.48       640
weighted avg       0.49      0.49      0.49       640



In [15]:
# Initialize counters for correct and incorrect predictions
correct_predictions = 0
incorrect_predictions = 0

# Loop through the validation data to count correct and incorrect predictions
for i, (images, true_label) in enumerate(val_fold):  # or test_dataset
    # Get the true label as an index
    true_label = np.argmax(true_label.numpy())  # Assuming one-hot encoded
    predicted_label = y_pred[i]  # Get predicted label

    # Check if the prediction is correct
    if true_label == predicted_label:
        correct_predictions += 1
    else:
        incorrect_predictions += 1

# Print the counts
print(f'Correct Predictions: {correct_predictions}')
print(f'Incorrect Predictions: {incorrect_predictions}')

# Optionally, print the total predictions and accuracy
total_predictions = correct_predictions + incorrect_predictions
accuracy = correct_predictions / total_predictions * 100 if total_predictions > 0 else 0
print(f'Total Predictions: {total_predictions}')
print(f'Accuracy: {accuracy:.2f}%')


Correct Predictions: 10
Incorrect Predictions: 10
Total Predictions: 20
Accuracy: 50.00%
